# Explore Delta Lake

In this notebook, you'll explore how to use Delta Lake in a Databricks Spark cluster.

## Ingest data

Use the **&#9656; Run Cell** menu option at the top-right of the following cell to run it and download a data file into the Databricks file system (DBFS).

In [0]:
%sh
rm -r /dbfs/data
rm -r /dbfs/delta
mkdir /dbfs/data
wget -O /dbfs/data/products.csv https://raw.githubusercontent.com/MicrosoftLearning/dp-203-azure-data-engineer/master/Allfiles/labs/25/data/products.csv

--2025-08-20 07:53:09--  https://raw.githubusercontent.com/MicrosoftLearning/dp-203-azure-data-engineer/master/Allfiles/labs/25/data/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14372 (14K) [text/plain]
Saving to: ‘/dbfs/data/products.csv’

     0K .......... ....                                       100% 2.30M=0.006s

2025-08-20 07:53:09 (2.30 MB/s) - ‘/dbfs/data/products.csv’ saved [14372/14372]



Now that you've ingested the data, you can load it into a Spark dataframe from the Databricks file system (DBFS)

In [0]:
df = spark.read.load('/data/products.csv', format='csv', header=True)
display(df.limit(10))

ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.9900
775,"Mountain-100 Black, 38",Mountain Bikes,3374.9900
776,"Mountain-100 Black, 42",Mountain Bikes,3374.9900
777,"Mountain-100 Black, 44",Mountain Bikes,3374.9900
778,"Mountain-100 Black, 48",Mountain Bikes,3374.9900
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.9900
780,"Mountain-200 Silver, 42",Mountain Bikes,2319.9900


## Load the file data into a delta table

You can persist the data in the dataframe in Delta format using the following code:

In [0]:
delta_table_path = "/delta/products-delta"
df.write.format("delta").save(delta_table_path)




The data for a delta lake table is stored in Parquet format. A log file is also created to track modifications made to the data.

Use the following shell commands to view the contents of the folder where the delta data has been saved.

In [0]:
%sh
ls /dbfs/delta/products-delta

_delta_log
part-00000-197c32ed-75dc-40a7-b8ea-2f412970c0ae.c000.snappy.parquet


The file data in Delta format can be loaded into a **DeltaTable** object, which you can use to view and update the data in the table. Run the following cell to update the data; reducing the price of product 771 by 10%.

In [0]:
from delta.tables import *
from pyspark.sql.functions import *

# Create a deltaTable object
deltaTable = DeltaTable.forPath(spark, delta_table_path)

# Update the table (reduce price of product 771 by 10%)
deltaTable.update(
    condition = "ProductID == 771",
    set = { "ListPrice": "ListPrice * 0.9" })

# View the updated data as a dataframe
deltaTable.toDF().show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      781|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows


The update is persisted to the data in the delta folder, and will be reflected in any new dataframe loaded from that location:

In [0]:
new_df = spark.read.format("delta").load(delta_table_path)
new_df.show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      781|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows


Data modifications are logged, enabling you to use the *time-travel* capabilities of Delta Lake to view previous versions of the data. For example, use the following code to view the original version of the product data:

In [0]:
new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
new_df.show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows


The log contains a full history of modifications to the data. Use the following code to see a record of the last 10 changes:

In [0]:
deltaTable.history(10).show(10, False, True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 1                                                                                                                                                                                                                                                                                                                           
 timestamp           | 2025-08-20 07:53:13                                                                                                                                                                                                                                                                                      

## Create catalog tables

So far you've worked with delta tables by loading data from the folder containing the parquet files on which the table is based. You can define *catalog tables* that encapsulate the data and provide a named table entity that you can reference in SQL code. Spark supports two kinds of catalog tables for delta lake:

- *External* tables that are defined by the path to the parquet files containing the table data.
- *Managed* tables, that are defined in the Hive metastore for the Spark cluster

### Create an external table

The following code creates a new database named **AdventureWorks** and then creates an external tabled named **ProductsExternal** in that database based on the path to the Delta files you defined previously.

In [0]:
spark.sql("CREATE DATABASE AdventureWorks")
spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-41736438718512>, line 1
----> 1 spark.sql("CREATE DATABASE AdventureWorks")
      2 spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
      3 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1868, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1863     else:
   1864         raise PySparkTypeError(
   18

Note that the **Location** property of the new table is the path you specified.

You can query the new table by using a SQL `SELECT` statement, like this:

In [0]:
%sql
USE AdventureWorks;

SELECT * FROM ProductsExternal;

### Create a managed table

A *managed* table stores its data files in the Hive metastore for the Spark cluster.

Run the following code to create (and then describe) a managed tabled named **ProductsManaged** based on the dataframe you originally loaded from the **products.csv** file (before you updated the price of product 771).

In [0]:
df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

You did not specify a path for the parquet files used by the table - this is managed for you in the Hive metastore, and shown in the **Location** property in the table description (in the **dbfs:/user/hive/warehouse/** path).

From the SQL user's perspective, there's no difference between external and managed tables when it comes to querying them with a `SELECT` statement:

In [0]:
%sql
USE AdventureWorks;

SELECT * FROM ProductsManaged;

### Compare external and managed tables

let's explore the differences between external and managed tables.

First, use the following code to list the tables in the **AdventureWorks** database:

In [0]:
%sql
USE AdventureWorks;

SHOW TABLES;

Now let's take a look at the folders on which these tables are based:

In [0]:
%sh
echo "External table:"
ls /dbfs/delta/products-delta
echo
echo "Managed table:"
ls /dbfs/user/hive/warehouse/adventureworks.db/productsmanaged

What happens if we use a `DROP` statement to delete these tables from the database?

In [0]:
%sql

USE AdventureWorks;

DROP TABLE IF EXISTS ProductsExternal;
DROP TABLE IF EXISTS ProductsManaged;

SHOW TABLES;

The metadata for both tables has been removed from the database; but what about the delta files?

In [0]:
%sh
echo "External table:"
ls /dbfs/delta/products-delta
echo
echo "Managed table:"
ls /dbfs/user/hive/warehouse/adventureworks.db/productsmanaged

The files for the managed table are deleted automatically when the table is dropped. However, the files for the external table remain. Dropping an external table only removes the table metadata from the database; it does not delete the data files.

You can use the following code to create a new table in the database that is based on the delta files in the **products-delta** folder:

In [0]:
%sql
USE AdventureWorks;

CREATE TABLE Products
USING DELTA
LOCATION '/delta/products-delta';

Now you can query the new table

In [0]:
%sql
USE AdventureWorks;

SELECT * FROM Products;

Because the table is based on the existing delta files, which include the logged history of changes, it reflects the modifications you previously made to the products data.

## Use delta tables for streaming data

Delta lake supports streaming data. Delta tables can be a *sink* or a *source* for data streams created using the Spark Structured Streaming API. In this example, you'll use a delta table as a sink for some streaming data in a simulated internet of things (IoT) scenario.

First, let's get some simulated device data in JSON format. Run the following cell to download a JSON file that looks like this:

```json
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
```

In [0]:
%sh
rm -r /dbfs/device_stream
mkdir /dbfs/device_stream
wget -O /dbfs/device_stream/devices1.json https://raw.githubusercontent.com/MicrosoftLearning/dp-203-azure-data-engineer/master/Allfiles/labs/25/data/devices1.json


Now you're ready to use Spark Structured Steraming to create a stream based on the folder containing the JSON device data.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a stream that reads data from the folder, using a JSON schema
inputPath = '/device_stream/'
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)
print("Source stream created...")

Now you'll take the stream of data you're reading from the folder, and perpetually write it to a delta table folder:

In [0]:
# Write the stream to a delta table
delta_stream_table_path = '/delta/iotdevicedata'
checkpointpath = '/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

To load the streamed table data, just read the delta table folder source like any other dataframe:

In [0]:
# Read the data in delta format into a dataframe
df = spark.read.format("delta").load(delta_stream_table_path)
display(df)

You can also create a table based on the streaming delta table folder:

In [0]:
# create a catalog table based on the streaming sink
spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

You can query the table just like any other:

In [0]:
%sql
SELECT * FROM IotDeviceData;

Now let's add some fresh device data to the stream.

In [0]:
%sh
wget -O /dbfs/device_stream/devices2.json https://raw.githubusercontent.com/MicrosoftLearning/dp-203-azure-data-engineer/master/Allfiles/labs/25/data/devices2.json

The new JSON data in the device folder is read into the stream and written to the delta folder, where it is reflected in the table:

In [0]:
%sql
SELECT * FROM IotDeviceData;

To stop the stream, use its **stop** method:

In [0]:
deltastream.stop()